In [1]:
### Imports

import numpy as np
from math import pi

In [2]:
### Auxiliary Functions for Number System Manipulations

def toBinary(x, bits):
    num = []
    while x > 1:
        rem = x % 2
        num.append(rem)
        x = int(x / 2)
    num.append(x)
    num = num[::-1]
    if(bits > len(num)):
        for i in range(bits - len(num)):
            num.insert(0, 0)
    return num

def toDecimal(x):
    x = np.array(x)
    x = x[::-1]
    n = np.size(x)
    value = np.sum(np.power(2, np.arange(n)) * x)
    return(value)

def bitreverser(x):
    x = np.array(x)
    N = np.size(x)
    indexes = np.arange(N)
    for i in range(N):
        indexes[i] = toDecimal(toBinary(indexes[i], int(np.log(N) / np.log(2)))[::-1])
    return(x[indexes])

In [3]:
### Auxiliary Functions for FFT Calculations

def X(m, n, N):
    return(np.exp(-(2 * pi) * 1j * m * n / N))

def X_inv(m, n, N):
    return(np.exp((2 * pi) * 1j * m * n / N))

def calculateButterFly_inv(a, b, bf, N):
    add = a + X_inv(bf ,1, N) * b
    sub = a - X_inv(bf, 1, N) * b
    return(add, sub)
    
def calculateButterFly(a, b, bf, N):
    add = a + X(bf ,1, N) * b
    sub = a - X(bf, 1, N) * b
    return(add, sub)

In [4]:
### Main FFT Code

def fft(x):
    N = np.size(x)
    stages = int(np.log(N) / np.log(2))
    values = bitreverser(np.array(x, dtype='complex'))
    for stage in range(stages):
        blocks = int( N / ( 2 ** (stage + 1) ) )
        butterflys = int( 2 ** stage )
        start = 0
        for block in range(blocks):
            for butterfly in range(butterflys):
                values[start + butterfly], values[start + butterfly + (2 ** stage)] = calculateButterFly(values[start + butterfly], values[start + butterfly + (2 ** stage)], butterfly, (2 ** (stage + 1)))
            start += int(N / blocks)
    return(values)            

In [5]:
### Main IFFT Code

def ifft(x):
    N = np.size(x)
    stages = int(np.log(N) / np.log(2))
    values = bitreverser(np.array(x, dtype='complex'))
    for stage in range(stages):
        blocks = int( N / ( 2 ** (stage + 1) ) )
        butterflys = int( 2 ** stage )
        start = 0
        for block in range(blocks):
            for butterfly in range(butterflys):
                values[start + butterfly], values[start + butterfly + (2 ** stage)] = calculateButterFly_inv(values[start + butterfly], values[start + butterfly + (2 ** stage)], butterfly, (2 ** (stage + 1)))
            start += int(N / blocks)
    return((1 / N) * values)            

In [6]:
### Testing it out

x = np.array([0., 0., 1., 0., 0., 0., 0., 0.])
y_test = np.round(fft(x))
y_actual = np.fft.fft(x)

In [7]:
np.allclose(y_test, y_actual)

True

In [8]:
np.round(ifft(y_test))

array([0.+0.j, 0.+0.j, 1.-0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j])